In [12]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from PIL import Image
import random, math, re, time, os, cv2, shutil, pandas

In [13]:
NUM_COLUMNAS = 200 # 200 siempre
NUM_TROZOS = 6
IMG_SIZE = 64

In [15]:
# Crear carpetas de las clases (dígitos)

training_directory = 'NEW-DATASET/Training'
validation_directory = 'NEW-DATASET/Validation'
testing_directory = 'NEW-DATASET/Testing'
 
"""for i in range(10): # 0 -> 9
    os.mkdir(f'{training_directory}/{i}')
    os.mkdir(f'{validation_directory}/{i}')
    os.mkdir(f'{testing_directory}/{i}')

for i in ['a','e','u']:
    os.mkdir(f'{training_directory}/{i}')
    os.mkdir(f'{validation_directory}/{i}')
    os.mkdir(f'{testing_directory}/{i}')"""

"for i in range(10): # 0 -> 9\n    os.mkdir(f'{training_directory}/{i}')\n    os.mkdir(f'{validation_directory}/{i}')\n    os.mkdir(f'{testing_directory}/{i}')\n\nfor i in ['a','e','u']:\n    os.mkdir(f'{training_directory}/{i}')\n    os.mkdir(f'{validation_directory}/{i}')\n    os.mkdir(f'{testing_directory}/{i}')"

# Mover GENÉRICO V3

In [16]:
archivoCSV1 = pandas.read_csv(filepath_or_buffer='train.csv', sep=',')
archivoCSV2 = pandas.read_csv(filepath_or_buffer='validation.csv', sep=',')

def trocear_labelV3(csv_file: str, index: int): # 'file.csv'
    if 'train' in csv_file:
        LABEL = archivoCSV1.iloc[index]['Label']
    else:
        LABEL = archivoCSV2.iloc[index]['Label']
    return [d for d in str(LABEL)]

In [17]:
def trocear_el_sushiV3(imagen: np.ndarray, num_trozos: int):
    lista_de_trocitos = []
    offset = int(NUM_COLUMNAS/num_trozos) # 200/6 = 33
    for trozo in range(num_trozos): 
        trozo_imagen = imagen[15:, trozo*offset:(trozo+1)*offset]
        trozo_imagen = Image.fromarray(trozo_imagen)
        trozo_imagen = trozo_imagen.resize(size=(IMG_SIZE,IMG_SIZE))
        trozo_imagen = np.array(trozo_imagen)
        lista_de_trocitos.append(trozo_imagen)
    return lista_de_trocitos

In [18]:
def repartir_trozos_a_carpetasV3(labels: list[int], trozos: list[np.ndarray], index_mega_lista: int, ruta: str, origin_directory: str):
    if origin_directory == 'train':
        marca = 'T'
    else: # 'validation'
        marca = 'V'
    for index, digito in enumerate(labels): # ej: [0, 2, 4, 7, 0, 6]
        trozos[index] = Image.fromarray(trozos[index]).save(f'{ruta}/{digito}/captcha{marca}_{index_mega_lista}_digito_nº{index+1}.png') 

In [19]:
def repartir_captchasV3(destination_directory: str, csv_file: str, origin_directory: str):
    for index, imagen in enumerate(os.listdir(origin_directory)):
        imagen = np.array(Image.open(f'{origin_directory}/{imagen}').resize((200,80)))

        label_troceado = trocear_labelV3(csv_file, index)
        label_troceado = [0] * (6-len(label_troceado)) + label_troceado
        imagen_troceada = trocear_el_sushiV3(imagen, NUM_TROZOS)

        # print(index, label_troceado)

        repartir_trozos_a_carpetasV3(labels=label_troceado, trozos=imagen_troceada, index_mega_lista=index, ruta=destination_directory, origin_directory=origin_directory)
        # if index==2:
        #     break

In [20]:
repartir_captchasV3(destination_directory=training_directory, csv_file='train.csv', origin_directory='train')
repartir_captchasV3(destination_directory=training_directory, csv_file='validation.csv', origin_directory='validation')

# Borrar todo el datset

In [33]:
ruta_destino = 'NEW-DATASET/Training'
for carpeta in os.listdir(ruta_destino):
    for imagen in os.listdir(f'{ruta_destino}/{carpeta}'):
        os.remove(f'{ruta_destino}/{carpeta}/{imagen}')

# Mover de Train a Val y Test y viceversa

In [21]:
directorio = "NEW-DATASET"

dir_train = f"{directorio}/Training"
dir_val = f"{directorio}/Validation"
dir_test = f"{directorio}/Testing"

lista_clases_train = os.listdir(dir_train)
lista_clases_val = os.listdir(dir_val)
lista_clases_test = os.listdir(dir_test)
#--------------------------------MOVER TODAS LAS FOTOS DE TESTING Y VALIDATION A TRAINING--------------------------------#
for i in range(len(lista_clases_train)):
    nombres_fotos_test = os.listdir(f'{dir_test}/{lista_clases_test[i]}')
    nombres_fotos_val = os.listdir(f'{dir_val}/{lista_clases_test[i]}')
    for foto_a_mover in nombres_fotos_test:
        shutil.move(src=f'{dir_test}/{lista_clases_test[i]}/{foto_a_mover}', dst=f'{dir_train}/{lista_clases_train[i]}')
    for foto_a_mover in nombres_fotos_val:
        shutil.move(src=f'{dir_val}/{lista_clases_test[i]}/{foto_a_mover}', dst=f'{dir_train}/{lista_clases_train[i]}')
print('TODO MOVIDO A TRAINING')
#--------------------------------MOVER TODAS LAS FOTOS DE TESTING Y VALIDATION A TRAINING--------------------------------#

#--------------------------------MOVER 10% FOTOS A TESTING Y 20% A VALIDATION--------------------------------#
for digito in lista_clases_train:
    nombres_fotos = os.listdir(f'{dir_train}/{digito}')
    cantidad_fotos_digito = len(nombres_fotos)

    diezmo = int(cantidad_fotos_digito * 0.1)
    diezmoX2 = int(cantidad_fotos_digito * 0.2)

    for i in range(diezmoX2): # para validación
        nombre_a_mover = nombres_fotos.pop(random.randrange(len(nombres_fotos))) 
        shutil.move(f'{dir_train}/{digito}/{nombre_a_mover}', f'{dir_val}/{digito}')
    for i in range(diezmo): # para test
        nombre_a_mover = nombres_fotos.pop(random.randrange(len(nombres_fotos))) 
        shutil.move(f'{dir_train}/{digito}/{nombre_a_mover}', f'{dir_test}/{digito}')
print('TODO MOVIDO A VALIDATION y TESTING')
#--------------------------------MOVER 10% FOTOS A TESTING Y 20% A VALIDATION--------------------------------#
# todo en menos de 20" 

TODO MOVIDO A VALIDATION y TESTING
